# Data Anaysis

## load data

In [3]:
from util.build_dataset import build_master

origin_df = build_master(dataset="origin_fmp", date="201908")

2020-04-04 14:50:27:manager:<INFO> 1 files is loaded from public_data/open_data_terrestrial_weather/origin/csv/2014-2020.csv in s3 'production-bobsim'
2020-04-04 14:50:28:manager:<INFO> 1 files is loaded from public_data/open_data_marine_weather/origin/csv/2014-2020.csv in s3 'production-bobsim'
2020-04-04 14:50:31:manager:<INFO> 1 files is loaded from public_data/open_data_raw_material_price/origin/csv/201908.csv in s3 'production-bobsim'


In [4]:
origin_df.head(5)

,조사일자,조사지역명,품목명,당일조사가격,평균기온(°C),최저기온(°C),최고기온(°C),강수 계속시간(hr),일강수량(mm),최대 풍속(m/s),...,평균기압(hPa),평균 상대습도(pct)_y,평균 기온(°C),평균 수온(°C),평균 최대 파고(m),평균 유의 파고(m),최고 유의 파고(m),최고 최대 파고(m),평균 파주기(sec),최고 파주기(sec)
0,2019-08-01,강원,감자감자수미수미,238.0,28.048422,24.331579,32.701054,4.1,11.83077,3.921053,...,1011.594116,91.0625,25.694117,24.729412,0.882353,0.529412,0.735294,1.252941,4.458824,5.547059
1,2019-08-01,강원,건제품건포도건포도(수입)수입,1450.0,28.048422,24.331579,32.701054,4.1,11.83077,3.921053,...,1011.594116,91.0625,25.694117,24.729412,0.882353,0.529412,0.735294,1.252941,4.458824,5.547059
2,2019-08-01,강원,고추붉은고추홍고추일반,990.0,28.048422,24.331579,32.701054,4.1,11.83077,3.921053,...,1011.594116,91.0625,25.694117,24.729412,0.882353,0.529412,0.735294,1.252941,4.458824,5.547059
3,2019-08-01,강원,고추풋고추꽈리고추꽈리고추,1050.0,28.048422,24.331579,32.701054,4.1,11.83077,3.921053,...,1011.594116,91.0625,25.694117,24.729412,0.882353,0.529412,0.735294,1.252941,4.458824,5.547059
4,2019-08-01,강원,고추풋고추청양청양고추,670.0,28.048422,24.331579,32.701054,4.1,11.83077,3.921053,...,1011.594116,91.0625,25.694117,24.729412,0.882353,0.529412,0.735294,1.252941,4.458824,5.547059


## weather data

### terrestrial weather

In [2]:
t_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214325 entries, 0 to 214324
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   일시            214325 non-null  datetime64[ns]
 1   평균기온(°C)      214086 non-null  float32       
 2   최저기온(°C)      214300 non-null  float32       
 3   최고기온(°C)      214305 non-null  float16       
 4   강수 계속시간(hr)   22288 non-null   float16       
 5   일강수량(mm)      77862 non-null   float16       
 6   최대 풍속(m/s)    214217 non-null  float16       
 7   평균 풍속(m/s)    214143 non-null  float16       
 8   최소 상대습도(pct)  214246 non-null  float16       
 9   평균 상대습도(pct)  213919 non-null  float32       
 10  합계 일조시간(hr)   213966 non-null  float16       
dtypes: datetime64[ns](1), float16(7), float32(3)
memory usage: 6.9 MB


In [3]:
t_df.isnull().sum()/214325*100

일시               0.000000
평균기온(°C)         0.111513
최저기온(°C)         0.011665
최고기온(°C)         0.009332
강수 계속시간(hr)     89.600840
일강수량(mm)        63.671060
최대 풍속(m/s)       0.050391
평균 풍속(m/s)       0.084918
최소 상대습도(pct)     0.036860
평균 상대습도(pct)     0.189432
합계 일조시간(hr)      0.167503
dtype: float64

In [28]:
t_dropped_df = t_df.drop('합계 일조시간(hr)', axis= 1)

In [39]:
import pandas as pd

def fillna_with_mean(df: pd.DataFrame):
    return df.interpolate(method='linear', limit_direction='both')

def fill_nan(x):
    columns_with_zero = ['강수 계속시간(hr)' , '일강수량(mm)']
    columns_with_mean = [
    '평균기온(°C)', '최저기온(°C)', '최고기온(°C)', 
    '최대 풍속(m/s)', '평균 풍속(m/s)', '최소 상대습도(pct)', '평균 상대습도(pct)'
    ]
    if x.name in columns_with_mean :
        return x.interpolate(method='linear', limit_direction='both')
    elif x.name in columns_with_zero:
        return x.fillna(0)
    else:
        return x

In [40]:
t_not_null_df = t_dropped_df.apply(fill_nan, axis=0)
t_not_null_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214325 entries, 0 to 214324
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   일시            214325 non-null  datetime64[ns]
 1   평균기온(°C)      214325 non-null  float32       
 2   최저기온(°C)      214325 non-null  float32       
 3   최고기온(°C)      214325 non-null  float16       
 4   강수 계속시간(hr)   214325 non-null  float16       
 5   일강수량(mm)      214325 non-null  float16       
 6   최대 풍속(m/s)    214325 non-null  float16       
 7   평균 풍속(m/s)    214325 non-null  float16       
 8   최소 상대습도(pct)  214325 non-null  float16       
 9   평균 상대습도(pct)  214325 non-null  float32       
dtypes: datetime64[ns](1), float16(6), float32(3)
memory usage: 6.5 MB
